# Error Bars

This code shows some information to describe how the last Mean Square Error of the Artificial Neural Networks variates when training several times. 

## Importing the libraries

In [1]:
import pybrain
from pybrain.structure import RecurrentNetwork, FeedForwardNetwork, LinearLayer, SigmoidLayer, FullConnection
from IPython.display import Image
from pybrain.structure import connections
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
from pybrain.datasets import SupervisedDataSet
from pybrain.supervised import BackpropTrainer
from pybrain.tools.customxml import NetworkWriter
from pybrain.tools.customxml import NetworkReader
%pylab inline --no-import-all


Populating the interactive namespace from numpy and matplotlib


## Defining the functions

The **Eval_FF**, **Eval_R** and **Eval_R2** functions train a network (Feedforward and Recurrent, respectively) with a *dataset*, in a determinated number of *epochs*, for an a given number of *samples*

In [2]:
def mse(predictions, targets):
    return ((predictions - targets) ** 2).mean()

In [3]:
def Eval_FF (net,data,s,t,epochs,samples):
    a = np.array([]) ## MSE array
            

    ##_______________________________________________________________________________________________________
    
    for n in range (0,samples):  ##Repeat the training 'samples' times
        
        ##____________________________________create the FF network__________________________________________
        netF = NetworkReader.readFrom(net)

        
        #____________________________________________Training____________________________________________
    
        trainer = BackpropTrainer(netF, data, learningrate = 0.001, momentum = 0.99, verbose=False)
        errors = np.array([])
        for epoch in range(0,epochs):
            temp = trainer.train()
            er2=np.array([])
            for inp in s:
                j=netF.activate(inp)
                er2=np.append(er2,j)
            er2=np.reshape(er2,[len(s),2])
            errors=np.append(errors,mse(er2,t))
            
        a= np.append(a,errors[epochs-1])
    return a

In [4]:
def Eval_R (net,data,s,t,epochs,samples):
    a = np.array([])
 
    ##_________________________________________________________________________________________________
    
    for n in range (0,samples):
        netR = NetworkReader.readFrom(net)       
        #____________________________________________Training____________________________________________
    
        trainer = BackpropTrainer(netR, data, learningrate = 0.001, momentum = 0.99, verbose=False)
        errors = np.array([])
        for epoch in range(0,epochs):
            netR.reset()
            temp = trainer.train()
            er2=np.array([])
            for inp in s:
                j=netF.activate(inp)
                er2=np.append(er2,j)
            er2=np.reshape(er2,[len(s),2])
            errors=np.append(errors,mse(er2,t))
            
        a= np.append(a,errors[epochs-1])
    return a

In [5]:
def Eval_R2 (net,data,s,t,epochs,samples):
    a = np.array([])

        
    ##________________________________________________________________________________________________
    
    for n in range (0,samples):
        netR = NetworkReader.readFrom(net)

        netR.maxoffset=2


        
        
        #____________________________________________Training____________________________________________     
    
        trainer = BackpropTrainer(netR, data, learningrate = 0.001, momentum = 0.99, verbose=False)
        errors = np.array([])
        for epoch in range(0,epochs):
            netR.reset()
            temp = trainer.train()
            er2=np.array([])
            for inp in s:
                j=netF.activate(inp)
                er2=np.append(er2,j)
            er2=np.reshape(er2,[len(s),2])
            errors=np.append(errors,mse(er2,t))
            
        a= np.append(a,errors2[epochs-1])
    return a

## Create the datasets

In [6]:
def getcsv( filename ):
    temp = np.fromfile( filename, sep=';' )
    numcols = len( temp )
    del( temp )
    df = pd.read_csv( filename, sep=';', names = [ x+1 for x in range(numcols)] )
    return df

In [7]:
train=getcsv('train.csv')
val = getcsv( 'val.csv' )
cols = [ \
    'Record(1)', 'Temperature(1)', 'RelHum(1)', \
    'Ventilation(1)', 'Screening(1)', 'Heating(1)', 'Cooling(1)', \
    'LAI(1)', 'OutTemp(1)', 'OutRelHum(1)', 'OutRad(1)', 'OutWindVel(1)', \
    'HourAngle(1)', 'Declination(1)', 'Elevation(1)', 'RadTheor(1)', \
    'Record(0)', 'Temperature(0)', 'RelHum(0)', \
    'Ventilation(0)', 'Screening(0)', 'Heating(0)', 'Cooling(0)', \
    'LAI(0)', 'OutTemp(0)', 'OutRelHum(0)', 'OutRad(0)', 'OutWindVel(0)', \
    'HourAngle(0)', 'Declination(0)', 'Elevation(0)', 'RadTheor(0)', \
    'Record(-1)', 'Temperature(-1)', 'RelHum(-1)', \
    'Ventilation(-1)', 'Screening(-1)', 'Heating(-1)', 'Cooling(-1)', \
    'LAI(-1)', 'OutTemp(-1)', 'OutRelHum(-1)', 'OutRad(-1)', 'OutWindVel(-1)', \
    'HourAngle(-1)', 'Declination(-1)', 'Elevation(-1)', 'RadTheor(-1)', \
    'Record(-2)', 'Temperature(-2)', 'RelHum(-2)', \
    'Ventilation(-2)', 'Screening(-2)', 'Heating(-2)', 'Cooling(-2)', \
    'LAI(-2)', 'OutTemp(-2)', 'OutRelHum(-2)', 'OutRad(-2)', 'OutWindVel(-2)', \
    'HourAngle(-2)', 'Declination(-2)', 'Elevation(-2)', 'RadTheor(-2)', \
    'Record(-3)', 'Temperature(-3)', 'RelHum(-3)', \
    'Ventilation(-3)', 'Screening(-3)', 'Heating(-3)', 'Cooling(-3)', \
    'LAI(-3)', 'OutTemp(-3)', 'OutRelHum(-3)', 'OutRad(-3)', 'OutWindVel(-3)', \
    'HourAngle(-3)', 'Declination(-3)', 'Elevation(-3)', 'RadTheor(-3)', \
    ]
train.columns = cols
val.columns = cols

## Data normalization

In [8]:
indexm=(["maximos","minimos"])
mm = pd.DataFrame(columns=train.columns,index=indexm)

for n in range(1,train.shape[1]):
    max1=max(train[train.columns[n]])
    max2=max(val[val.columns[n]])
    mm[mm.columns[n]]["maximos"]=max(max1,max2)
    
    min1=min(train[train.columns[n]])
    min2=min(val[val.columns[n]])
    mm[mm.columns[n]]["minimos"]=min(min1,min2)


In [9]:
def norm(x,maxi,mini):
    Vnorm=(2*((x-mini)/(maxi-mini)))-1
    return Vnorm  


In [10]:
Ntrain = pd.DataFrame(columns=train.columns)
for n in range(1,train.shape[1]):
    ar=train[train.columns[n]]
    Ntrain[Ntrain.columns[n]]=norm(ar,mm[mm.columns[n]]["maximos"],mm[mm.columns[n]]["minimos"])

In [11]:
s=np.array([Ntrain['Temperature(-2)'],Ntrain['RelHum(-2)'],Ntrain['Ventilation(-2)'],Ntrain['Screening(-2)'], \
           Ntrain['HourAngle(-2)'],Ntrain['Declination(-2)'],Ntrain['Elevation(-2)'],Ntrain['RadTheor(-2)'], \
           Ntrain['Temperature(-1)'],Ntrain['RelHum(-1)'],Ntrain['Ventilation(-1)'],Ntrain['Screening(-1)'], \
           Ntrain['HourAngle(-1)'],Ntrain['Declination(-1)'],Ntrain['Elevation(-1)'],Ntrain['RadTheor(-1)'], \
           Ntrain['Temperature(0)'],Ntrain['RelHum(0)'],Ntrain['Ventilation(0)'],Ntrain['Screening(0)'], \
           Ntrain['HourAngle(0)'],Ntrain['Declination(0)'],Ntrain['Elevation(0)'],Ntrain['RadTheor(0)']]).T

t=np.array([Ntrain['Temperature(1)'],Ntrain['RelHum(1)']]).T



In [12]:
sR=np.array([Ntrain['Temperature(0)'],Ntrain['RelHum(0)'],Ntrain['Ventilation(0)'],Ntrain['Screening(0)'], \
           Ntrain['HourAngle(0)'],Ntrain['Declination(0)'],Ntrain['Elevation(0)'],Ntrain['RadTheor(0)']]).T

tR=np.array([Ntrain['Temperature(1)'],Ntrain['RelHum(1)']]).T


In [13]:
#a=len(s)
a=50

In [14]:
dataF = SupervisedDataSet(24,2)
dataR = SupervisedDataSet(8,2)

for m in range(0,a):
    dataF.addSample(s[m],t[m])  
    dataR.addSample(sR[m],tR[m])

## Evaluating the Networks

In [ ]:
## Number of epochs
n=50

## Number of Samples
m=10

In [ ]:
F=Eval_FF ('NET_FF.xml',dataF,s,t,n,m) ##For n epochs, training m times
R=Eval_R ('NET_R.xml',dataR,sR,tR,n,m) ##For n epochs, training m times
R2=Eval_R2 ('NET_R2.xml',dataR,sR,tR,n,m) ##For n epochs, training m times
Errors=([F, R, R2])

## Calculating the Mean

The sum of the data points divided by the number data points
$$M=\frac{\sum_{i=0}^n (x_i)}{n}$$

In [ ]:
MEAN=np.array([])
for n in range(0,len(Errors)):
    MEAN=np.append(MEAN,Errors[n].mean())

## Calculating the Standar Deviation

Average difference between the data points and their mean:

$$SD=\sqrt{\sum \frac{(x-M)^2}{n-1}}$$

In [ ]:
Sd=np.array([])
for n in range(0,len(Errors)):
    Sd=np.append(Sd,Errors[n].std())

In [ ]:
fig, ax = plt.subplots(1, sharex=True, figsize=(10,7))
a=len(Errors)
ind = np.arange(a)
width = 0.35 # the width of the bars
mean = ax.bar(ind+width, MEAN, width, color='green',yerr=Sd, label='Mean Square Error')
measurements=ax.plot(ind+0.9*width,Errors,'ro',color='blue', label='Last error')
ax.set_xticks(ind+1.5*width)
ax.set_xticklabels(('FeedForward',  'Recurrent', 'Recurrent 2'))

blue_line = mlines.Line2D([], [], color='blue', marker='|',
                          markersize=8, label='Standar Deviation')
blue_point = mlines.Line2D([], [], color='blue', marker='o',
                          markersize=8, label='Last training error')
plt.legend(handles=[mean,blue_line,blue_point])

ax.set_ylabel('Mean Square Error')
ax.set_title('Descriptive Error Bars for 20 trainings',size='xx-large')
plt.show()

In [ ]:
5